# k-Nearest Neighbors (k-NN) and Instance-Based Learning

## Overview
This notebook provides a comprehensive analysis of the k-Nearest Neighbors algorithm, including:
- Algorithm intuition and mathematical foundation
- Implementation from scratch
- Testing on real datasets
- Comparison with other models
- Analysis of lazy learning concepts
- Distance metrics evaluation
- Performance on high-dimensional datasets

## 1. Algorithm Intuition and Mathematics

### What is k-NN?
k-Nearest Neighbors is a **lazy learning** algorithm that:
- Stores all training data during the training phase
- Makes predictions based on the k closest training examples
- Uses majority voting for classification
- Uses averaging for regression

### Mathematical Foundation

**Distance Metrics:**
1. **Euclidean Distance**: $d(x,y) = \sqrt{\sum_{i=1}^{n}(x_i - y_i)^2}$
2. **Manhattan Distance**: $d(x,y) = \sum_{i=1}^{n}|x_i - y_i|$
3. **Cosine Distance**: $d(x,y) = 1 - \frac{x \cdot y}{||x|| \cdot ||y||}$

**Prediction Formula:**
For classification: $\hat{y} = \text{mode}(y_{k-neighbors})$

**Key Characteristics:**
- **Lazy Learning**: No explicit training phase
- **Instance-based**: Uses specific instances for prediction
- **Non-parametric**: Makes no assumptions about data distribution
- **Curse of Dimensionality**: Performance degrades in high dimensions

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

In [ ]:
# Import our custom k-NN implementation
import sys
sys.path.append('../src')
from knn_implementation import KNearestNeighbors, load_and_prepare_data

## 2. Dataset Loading and Exploration

## 2. Real Dataset Analysis

Now let's load and analyze the real datasets we downloaded from various sources. These datasets will help us understand how k-NN performs on different types of problems.

In [ ]:
# Load the downloaded real datasets
import os

# Check if data directory exists and list available datasets
data_dir = '../data'
if os.path.exists(data_dir):
    print("Available datasets:")
    for file in os.listdir(data_dir):
        if file.endswith('.csv'):
            print(f"  - {file}")
else:
    print("Data directory not found. Please run the dataset downloader first.")

# Load each dataset
real_datasets = {}

try:
    # Banknote Authentication Dataset
    banknote_df = pd.read_csv('../data/banknote_authentication.csv')
    real_datasets['Banknote Authentication'] = {
        'data': banknote_df,
        'target_col': 'class',
        'description': 'Distinguish genuine vs forged banknotes based on image features'
    }
    print(f"\nBanknote Authentication: {banknote_df.shape[0]} samples, {banknote_df.shape[1]-1} features")
    
    # Glass Identification Dataset
    glass_df = pd.read_csv('../data/glass_identification.csv')
    real_datasets['Glass Identification'] = {
        'data': glass_df,
        'target_col': 'glass_type',
        'description': 'Classify glass types based on chemical composition'
    }
    print(f"Glass Identification: {glass_df.shape[0]} samples, {glass_df.shape[1]-1} features")
    
    # Ionosphere Dataset
    ionosphere_df = pd.read_csv('../data/ionosphere.csv')
    real_datasets['Ionosphere'] = {
        'data': ionosphere_df,
        'target_col': 'class',
        'description': 'Radar data classification (good vs bad returns)'
    }
    print(f"Ionosphere: {ionosphere_df.shape[0]} samples, {ionosphere_df.shape[1]-1} features")
    
    # Synthetic High-Dimensional Dataset
    synthetic_df = pd.read_csv('../data/synthetic_high_dim.csv')
    real_datasets['Synthetic High-Dim'] = {
        'data': synthetic_df,
        'target_col': 'target',
        'description': 'High-dimensional synthetic data for curse of dimensionality analysis'
    }
    print(f"Synthetic High-Dim: {synthetic_df.shape[0]} samples, {synthetic_df.shape[1]-1} features")
    
except FileNotFoundError as e:
    print(f"Error loading datasets: {e}")
    print("Please ensure the dataset downloader has been run successfully.")

print(f"\nSuccessfully loaded {len(real_datasets)} real datasets!")

In [ ]:
# Visualize dataset characteristics
if real_datasets:
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Real Datasets Overview', fontsize=16)
    
    dataset_names = list(real_datasets.keys())
    
    for i, (name, dataset_info) in enumerate(real_datasets.items()):
        if i >= 4:  # Only plot first 4 datasets
            break
            
        row = i // 2
        col = i % 2
        
        df = dataset_info['data']
        target_col = dataset_info['target_col']
        
        # Plot class distribution
        class_counts = df[target_col].value_counts()
        axes[row, col].bar(range(len(class_counts)), class_counts.values)
        axes[row, col].set_title(f'{name}\nClass Distribution')
        axes[row, col].set_xlabel('Class')
        axes[row, col].set_ylabel('Count')
        axes[row, col].set_xticks(range(len(class_counts)))
        axes[row, col].set_xticklabels(class_counts.index)
        
        # Add dataset info as text
        info_text = f"Samples: {df.shape[0]}\nFeatures: {df.shape[1]-1}\nClasses: {len(class_counts)}"
        axes[row, col].text(0.02, 0.98, info_text, transform=axes[row, col].transAxes,
                           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    # Print detailed statistics
    print("\nDetailed Dataset Statistics:")
    print("=" * 50)
    for name, dataset_info in real_datasets.items():
        df = dataset_info['data']
        target_col = dataset_info['target_col']
        print(f"\n{name}:")
        print(f"  Description: {dataset_info['description']}")
        print(f"  Shape: {df.shape}")
        print(f"  Target distribution: {dict(df[target_col].value_counts())}")
        print(f"  Missing values: {df.isnull().sum().sum()}")

## 3. Comprehensive k-NN Evaluation on Real Datasets

Let's evaluate our k-NN implementation on all the real datasets and compare performance across different data characteristics.

In [ ]:
# Import dataset preparation function
sys.path.append('../src')
from dataset_downloader import prepare_dataset_for_knn

# Evaluate k-NN on all real datasets
real_dataset_results = {}
print("k-NN Evaluation on Real Datasets")
print("=" * 50)

for name, dataset_info in real_datasets.items():
    print(f"\nEvaluating {name}...")
    
    df = dataset_info['data']
    target_col = dataset_info['target_col']
    
    try:
        # Prepare data
        X_train, X_test, y_train, y_test, feature_names = prepare_dataset_for_knn(
            df, target_col, test_size=0.3
        )
        
        # Test different k values
        k_values = range(1, 16)
        accuracies = []
        
        for k in k_values:
            # Use our custom k-NN implementation
            knn = KNearestNeighbors(k=k)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            accuracies.append(acc)
        
        # Store results
        best_k = k_values[np.argmax(accuracies)]
        best_accuracy = max(accuracies)
        
        real_dataset_results[name] = {
            'k_values': k_values,
            'accuracies': accuracies,
            'best_k': best_k,
            'best_accuracy': best_accuracy,
            'n_samples': len(df),
            'n_features': len(feature_names)
        }
        
        print(f"  Best k: {best_k}, Best accuracy: {best_accuracy:.4f}")
        
    except Exception as e:
        print(f"  Error processing {name}: {e}")

print(f"\nEvaluation completed for {len(real_dataset_results)} datasets!")

In [ ]:
# Visualize k-NN performance across all real datasets
if real_dataset_results:
    # Plot 1: k vs Accuracy for all datasets
    plt.figure(figsize=(15, 10))
    
    # Subplot 1: Individual dataset performance
    plt.subplot(2, 2, 1)
    for name, results in real_dataset_results.items():
        plt.plot(results['k_values'], results['accuracies'], 
                marker='o', label=name, linewidth=2)
    plt.title('k-NN Performance vs k Value (All Real Datasets)')
    plt.xlabel('k Value')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Subplot 2: Best accuracy comparison
    plt.subplot(2, 2, 2)
    names = list(real_dataset_results.keys())
    best_accs = [real_dataset_results[name]['best_accuracy'] for name in names]
    bars = plt.bar(names, best_accs, color=plt.cm.Set3(np.linspace(0, 1, len(names))))
    plt.title('Best k-NN Accuracy by Dataset')
    plt.ylabel('Best Accuracy')
    plt.xticks(rotation=45)
    
    # Add value labels on bars
    for bar, acc in zip(bars, best_accs):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{acc:.3f}', ha='center', va='bottom')
    
    # Subplot 3: Dataset complexity analysis
    plt.subplot(2, 2, 3)
    n_samples = [real_dataset_results[name]['n_samples'] for name in names]
    n_features = [real_dataset_results[name]['n_features'] for name in names]
    
    scatter = plt.scatter(n_features, best_accs, s=[s/5 for s in n_samples], 
                         c=range(len(names)), cmap='viridis', alpha=0.7)
    plt.xlabel('Number of Features')
    plt.ylabel('Best Accuracy')
    plt.title('Accuracy vs Dataset Complexity')
    
    # Add dataset labels
    for i, name in enumerate(names):
        plt.annotate(name, (n_features[i], best_accs[i]), 
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # Subplot 4: Optimal k distribution
    plt.subplot(2, 2, 4)
    optimal_ks = [real_dataset_results[name]['best_k'] for name in names]
    plt.hist(optimal_ks, bins=range(1, max(optimal_ks)+2), alpha=0.7, edgecolor='black')
    plt.title('Distribution of Optimal k Values')
    plt.xlabel('Optimal k')
    plt.ylabel('Frequency')
    plt.xticks(range(1, max(optimal_ks)+1))
    
    plt.tight_layout()
    plt.show()
    
    # Summary table
    print("\nSummary Table - k-NN Performance on Real Datasets:")
    print("=" * 80)
    print(f"{'Dataset':<20} {'Samples':<8} {'Features':<9} {'Best k':<7} {'Best Acc':<10} {'Complexity':<12}")
    print("-" * 80)
    
    for name, results in real_dataset_results.items():
        complexity = "High" if results['n_features'] > 20 else "Medium" if results['n_features'] > 10 else "Low"
        print(f"{name:<20} {results['n_samples']:<8} {results['n_features']:<9} {results['best_k']:<7} {results['best_accuracy']:<10.4f} {complexity:<12}")

In [ ]:
## 4. Built-in Datasets Analysis

Let's also analyze some built-in scikit-learn datasets for comparison.

# Load multiple datasets for comprehensive testing
datasets = {
    'Iris': load_iris(),
    'Wine': load_wine(),
    'Breast Cancer': load_breast_cancer()
}

# Display dataset information
for name, dataset in datasets.items():
    print(f"\n{name} Dataset:")
    print(f"  Samples: {dataset.data.shape[0]}")
    print(f"  Features: {dataset.data.shape[1]}")
    print(f"  Classes: {len(dataset.target_names)}")
    print(f"  Class names: {dataset.target_names}")

In [ ]:
# Detailed analysis of Iris dataset
iris = load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target
iris_df['species'] = iris_df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

print("Iris Dataset Summary:")
print(iris_df.describe())

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Pairplot for first two features
sns.scatterplot(data=iris_df, x='sepal length (cm)', y='sepal width (cm)', 
                hue='species', ax=axes[0,0])
axes[0,0].set_title('Sepal Length vs Width')

# Pairplot for last two features
sns.scatterplot(data=iris_df, x='petal length (cm)', y='petal width (cm)', 
                hue='species', ax=axes[0,1])
axes[0,1].set_title('Petal Length vs Width')

# Distribution of features
iris_df[iris.feature_names].hist(bins=20, ax=axes[1,0], alpha=0.7)
axes[1,0].set_title('Feature Distributions')

# Correlation heatmap
sns.heatmap(iris_df[iris.feature_names].corr(), annot=True, ax=axes[1,1])
axes[1,1].set_title('Feature Correlations')

plt.tight_layout()
plt.show()

## 5. k-NN Implementation and Testing on Iris Dataset

In [ ]:
# Prepare data for k-NN analysis
X_train, X_test, y_train, y_test, feature_names, target_names = load_and_prepare_data()

# Test our custom k-NN implementation
print("Testing Custom k-NN Implementation")
print("=" * 40)

knn_custom = KNearestNeighbors(k=5)
knn_custom.fit(X_train, y_train)
y_pred_custom = knn_custom.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_custom)
print(f"Custom k-NN Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_custom, target_names=target_names))

## 6. Hyperparameter Tuning - Finding Optimal k

In [ ]:
# Test different k values
k_values = range(1, 21)
train_accuracies = []
test_accuracies = []

for k in k_values:
    # Custom implementation
    knn = KNearestNeighbors(k=k)
    knn.fit(X_train, y_train)
    
    # Train accuracy
    y_pred_train = knn.predict(X_train)
    train_acc = accuracy_score(y_train, y_pred_train)
    train_accuracies.append(train_acc)
    
    # Test accuracy
    y_pred_test = knn.predict(X_test)
    test_acc = accuracy_score(y_test, y_pred_test)
    test_accuracies.append(test_acc)

# Plot results
plt.figure(figsize=(12, 6))
plt.plot(k_values, train_accuracies, marker='o', label='Training Accuracy', linewidth=2)
plt.plot(k_values, test_accuracies, marker='s', label='Testing Accuracy', linewidth=2)
plt.title('k-NN Performance: Training vs Testing Accuracy')
plt.xlabel('k Value')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(k_values)

# Highlight best k for test accuracy
best_k = k_values[np.argmax(test_accuracies)]
plt.axvline(x=best_k, color='red', linestyle='--', alpha=0.7, 
            label=f'Best k = {best_k}')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Best k value: {best_k} with test accuracy: {max(test_accuracies):.4f}")

## 7. Distance Metrics Comparison

In [ ]:
# Compare different distance metrics
metrics = ['euclidean', 'manhattan', 'cosine']
metric_results = {}

plt.figure(figsize=(15, 5))

for i, metric in enumerate(metrics):
    # Test with different k values
    k_range = range(1, 16)
    accuracies = []
    
    for k in k_range:
        knn = KNearestNeighbors(k=k, distance_metric=metric)
        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)
    
    metric_results[metric] = max(accuracies)
    
    # Plot
    plt.subplot(1, 3, i+1)
    plt.plot(k_range, accuracies, marker='o', linewidth=2)
    plt.title(f'{metric.capitalize()} Distance')
    plt.xlabel('k Value')
    plt.ylabel('Accuracy')
    plt.grid(True, alpha=0.3)
    plt.ylim(0.8, 1.05)

plt.tight_layout()
plt.show()

# Summary
print("Distance Metric Comparison (Best Accuracy):")
for metric, acc in metric_results.items():
    print(f"{metric.capitalize()}: {acc:.4f}")

## 8. Comparison with Other Machine Learning Models

In [ ]:
# Compare k-NN with other popular algorithms
models = {
    'k-NN (k=5)': KNeighborsClassifier(n_neighbors=5),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', random_state=42)
}

model_results = {}
predictions = {}

print("Model Comparison Results:")
print("=" * 40)

for name, model in models.items():
    # Train and predict
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    model_results[name] = accuracy
    predictions[name] = y_pred
    
    print(f"{name}: {accuracy:.4f}")

# Visualize comparison
plt.figure(figsize=(10, 6))
models_list = list(model_results.keys())
accuracies_list = list(model_results.values())

bars = plt.bar(models_list, accuracies_list, color=['skyblue', 'lightgreen', 'salmon'])
plt.title('Model Performance Comparison')
plt.ylabel('Accuracy')
plt.ylim(0.8, 1.05)

# Add value labels on bars
for bar, acc in zip(bars, accuracies_list):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 9. High-Dimensional Dataset Analysis

In [ ]:
# Test on high-dimensional dataset (Breast Cancer - 30 features)
cancer = load_breast_cancer()
X_cancer, y_cancer = cancer.data, cancer.target

print(f"Breast Cancer Dataset:")
print(f"Samples: {X_cancer.shape[0]}, Features: {X_cancer.shape[1]}")
print(f"Classes: {cancer.target_names}")

# Split and scale
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test_split(
    X_cancer, y_cancer, test_size=0.3, random_state=42, stratify=y_cancer
)

scaler = StandardScaler()
X_train_cancer_scaled = scaler.fit_transform(X_train_cancer)
X_test_cancer_scaled = scaler.transform(X_test_cancer)

# Test k-NN performance on high-dimensional data
k_values = [1, 3, 5, 7, 9, 11, 15, 20]
cancer_accuracies = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_cancer_scaled, y_train_cancer)
    y_pred = knn.predict(X_test_cancer_scaled)
    acc = accuracy_score(y_test_cancer, y_pred)
    cancer_accuracies.append(acc)
    print(f"k={k}: Accuracy = {acc:.4f}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(k_values, cancer_accuracies, marker='o', linewidth=2, markersize=8)
plt.title('k-NN Performance on High-Dimensional Data (30 features)')
plt.xlabel('k Value')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
best_k_cancer = k_values[np.argmax(cancer_accuracies)]
plt.axvline(x=best_k_cancer, color='red', linestyle='--', alpha=0.7, 
            label=f'Best k = {best_k_cancer}')
plt.legend()
plt.tight_layout()
plt.show()

## 10. Curse of Dimensionality Demonstration

In [ ]:
# Demonstrate curse of dimensionality
from sklearn.datasets import make_classification

n_samples = 1000
n_features_list = [2, 5, 10, 20, 50, 100]
dimensionality_results = []

print("Curse of Dimensionality Analysis:")
print("=" * 35)

for n_features in n_features_list:
    # Generate synthetic dataset
    X, y = make_classification(n_samples=n_samples, n_features=n_features, 
                              n_informative=min(n_features, 10), 
                              n_redundant=0, n_clusters_per_class=1, 
                              random_state=42)
    
    # Split and scale
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Test k-NN
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_scaled, y_train)
    y_pred = knn.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    
    dimensionality_results.append(accuracy)
    print(f"Features: {n_features:3d}, Accuracy: {accuracy:.4f}")

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(n_features_list, dimensionality_results, marker='o', linewidth=2, markersize=8)
plt.title('k-NN Performance vs. Number of Features (Curse of Dimensionality)')
plt.xlabel('Number of Features')
plt.ylabel('Accuracy')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 11. Cross-Validation Analysis

In [ ]:
# Perform cross-validation for robust evaluation
from sklearn.model_selection import cross_val_score

# Test different k values with cross-validation
k_values = range(1, 21)
cv_means = []
cv_stds = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy')
    cv_means.append(scores.mean())
    cv_stds.append(scores.std())

# Plot with error bars
plt.figure(figsize=(12, 6))
plt.errorbar(k_values, cv_means, yerr=cv_stds, marker='o', linewidth=2, 
             capsize=5, capthick=2)
plt.title('k-NN Cross-Validation Performance')
plt.xlabel('k Value')
plt.ylabel('Cross-Validation Accuracy')
plt.grid(True, alpha=0.3)
plt.xticks(k_values)

# Find and highlight best k
best_k_cv = k_values[np.argmax(cv_means)]
plt.axvline(x=best_k_cv, color='red', linestyle='--', alpha=0.7, 
            label=f'Best k = {best_k_cv}')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Best k from cross-validation: {best_k_cv}")
print(f"CV Accuracy: {cv_means[best_k_cv-1]:.4f} ± {cv_stds[best_k_cv-1]:.4f}")

## 12. Summary and Key Findings

In [ ]:
# Create comprehensive summary
print("k-NN ANALYSIS SUMMARY")
print("=" * 50)
print("\n1. ALGORITHM CHARACTERISTICS:")
print("   • Lazy Learning: No training phase, stores all data")
print("   • Instance-based: Uses specific training examples")
print("   • Non-parametric: No assumptions about data distribution")
print("   • Memory-intensive: Stores entire training set")

print("\n2. DISTANCE METRICS PERFORMANCE:")
for metric, acc in metric_results.items():
    print(f"   • {metric.capitalize()}: {acc:.4f}")

print("\n3. MODEL COMPARISON:")
for model, acc in model_results.items():
    print(f"   • {model}: {acc:.4f}")

print(f"\n4. OPTIMAL HYPERPARAMETERS:")
print(f"   • Best k (simple): {best_k}")
print(f"   • Best k (CV): {best_k_cv}")
print(f"   • Best distance metric: {max(metric_results.items(), key=lambda x: x[1])[0]}")

print("\n5. HIGH-DIMENSIONAL PERFORMANCE:")
print(f"   • Best accuracy on 30D data: {max(cancer_accuracies):.4f}")
print(f"   • Performance degrades with increasing dimensions")

print("\n6. KEY INSIGHTS:")
print("   • k-NN works well on low-dimensional, well-separated data")
print("   • Sensitive to curse of dimensionality")
print("   • Feature scaling is crucial for distance-based algorithms")
print("   • Choice of k affects bias-variance tradeoff")
print("   • Computationally expensive at prediction time")